# Listwise Approach

In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pickle

In [0]:
class Listwise(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(Listwise, self).__init__();
    ''' Elman RNN for overall context of the set of document vectors that needs to be chronologically ordered. '''
    self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
    
    ''' MLP that computes the final scores for each document vector. '''
    self.mlp = nn.Sequential(
        nn.Linear(2*hidden_size, 64),
        nn.ReLU(),
        nn.Linear(64,64),
        nn.ReLU(),
        nn.Linear(64, 1),
        nn.Sigmoid()
    )
  
  def forward(self, input_doc_vectors):
    ''' input_doc_vectors: a list of document vectors(tensors) that needs to be chronologically arranged. '''
    outputs = []
    for i, doc_vec in enumerate(input_doc_vectors):
      # print("doc vec shape: ", doc_vec.shape)
      doc_vec_in = doc_vec.view(1,1,doc_vec.shape[0])
      # print("doc vec in shape: ", doc_vec_in.shape)
      if i == 0:
        output, h_next = self.rnn(doc_vec_in)
      else:
        output, h_next = self.rnn(doc_vec_in, h_next)
      outputs.append(output.view(output.shape[2]))
    
    final_hidden_state = h_next.view(h_next.shape[2])
    # print("final hidden state shape: ", final_hidden_state.shape)
    doc_vec_scores = []

    for out in outputs:
      mlp_in = torch.cat([out, final_hidden_state]).unsqueeze(0)
      doc_vec_score = self.mlp(mlp_in)[0]
      doc_vec_scores.append(doc_vec_score)
    
    doc_vec_scores_tensor = torch.stack(doc_vec_scores)
    return doc_vec_scores_tensor
  
  def train(self, X, y, optimizer, device, epochs=10, batch_size=20):
    batch_no = 1
    for ep in range(epochs):
      print("epoch no: ", ep)
      losses = []
      for i, (doc_vec_set, target) in enumerate(zip(X, y)):
        doc_vec_tensor_set = [torch.from_numpy(x).float().to(device) for x in doc_vec_set]
        target_tensor = torch.tensor(target).float().to(device)
        
        predicted_scores_tensor = self(doc_vec_tensor_set)
        # print(target_tensor.view(target_tensor.shape[0], 1).shape, predicted_scores_tensor.shape)
        
        loss = nn.MSELoss()(predicted_scores_tensor, target_tensor.view(target_tensor.shape[0], 1))
        losses.append(loss)
        if (i+1) % batch_size == 0:
          optimizer.zero_grad()
          mean_loss = torch.stack(losses).mean()
          mean_loss.backward()
          optimizer.step()
          losses = []
          print("batch no. "+str(batch_no)+" mean loss: ", mean_loss)
          batch_no += 1
  
  def inference(self, X):
    ''' X: a list in which each element is a list of document vectors that needs to be chronologically arranged. '''
    outs = []
    for doc_vec_set in X:
      doc_vec_tensor_set = [torch.from_numpy(x).float().to(device) for x in doc_vec_set]
      predicted_scores_tensor = self(doc_vec_tensor_set)
      # print(predicted_scores_tensor, predicted_scores_tensor.shape)
      predicted_order_of_documents = predicted_scores_tensor.argsort()
      outs.append(predicted_order_of_documents.detach().cpu().numpy())
    if len(outs) == 1:  return outs[0]
    return outs

def save_model(path, model):
  torch.save(model.state_dict(), path)

def load_model(path, model):
  state_dict = torch.load(path)
  model.load_state_dict(state_dict)

In [0]:
def load_pickle(path):
  with open(path, "rb") as f:
    data = pickle.load(f)
  return data

def save_pickle(path, data):
  with open(path, "wb") as f:
    pickle.dump(data, f)

In [0]:
path_to_ir = "/content/drive/My Drive/IR Project DATA"

In [0]:
dataset = load_pickle(os.path.join(path_to_ir, "XXX"))

In [ ]:
X, y = dataset
print(len(X))
print(X[0][0].shape)
print(y[0])

In [0]:
Xtrain, ytrain = X[:1500], y[:1500]
Xtest, ytest = X[1500:], y[1500:]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
input_size=X[0][0].shape[0]
hidden_size=input_size

In [0]:
model = Listwise(input_size=input_size, hidden_size=hidden_size).to(device)

In [ ]:
model.train(Xtrain, ytrain, optim.Adam(model.parameters(), lr=0.001), device, epochs=20, batch_size=100)

In [0]:
save_model(os.path.join(path_to_ir, "model1"), model)